In [1]:
import os
import astropy.units as u
import warnings
from sunpy.net import Fido,attrs
from datetime import date, time, datetime, timedelta
from astropy.io import fits

# Data file generator (title)

### Author: Alex Feghhi 
### Created: 7/30/2019
### Last run: 8/01/2019 

### Goals:
- Downloads sharp files from a given date range and extracts and stores given keywords into a textfile

Generate work directory and sharp storage directory

In [2]:
workdir = 'C:/Users/alexf/Desktop/HMI_Data/'
sharp_dir = workdir + 'sharp/'

if not os.path.exists(workdir):
    os.mkdir(workdir)
    print("Directory " + workdir + "does not exist. Creating...")

if not os.path.exists(sharp_dir):
    os.mkdir(sharp_dir)
    print("Directory " + sharp_dir + "does not exist. Creating...")

Define start,end time, time interval between harps, and download chunk size

In [3]:
start = datetime(2011, 10, 10,0,0,0)#date time object format is year, month, day, hour, minute, second
end = datetime(2012, 5, 1,0,0,0)#currently generating a year of data
time_interval = timedelta(minutes = 60)
download_chunk = timedelta(days = 10)#avoid download chunks greater than 1 month in order to not download too much at once

## This Cell downloads all sharp files in the given time range from the jsoc server. Only run once.

In [4]:
current_time = start
while(current_time < end):
    if(end-current_time > download_chunk):
        next_time = current_time + download_chunk
    else:
        next_time = end
    response = Fido.search(
    attrs.jsoc.Time(current_time, next_time),
    attrs.jsoc.Notify('hsmgroupnasa@gmail.com'),
    attrs.jsoc.Series('hmi.Sharp_720s'),
    attrs.jsoc.Segment('bitmap'),
    attrs.Sample(time_interval.total_seconds() * u.s)
    )
    response
    res = Fido.fetch(response, path= sharp_dir + '/{file}.fits')
    current_time = next_time

Export request pending. [id="JSOC_20190811_276", status=2]
Waiting for 0 seconds...
Request not found on server, 5 retries left.
Export request pending. [id="JSOC_20190811_276", status=6]
Waiting for 5 seconds...
Request not found on server, 4 retries left.
Export request pending. [id="JSOC_20190811_276", status=6]
Waiting for 5 seconds...
Export request pending. [id="JSOC_20190811_276", status=1]
Waiting for 5 seconds...
Export request pending. [id="JSOC_20190811_276", status=1]
Waiting for 5 seconds...
Export request pending. [id="JSOC_20190811_276", status=1]
Waiting for 5 seconds...
Export request pending. [id="JSOC_20190811_276", status=1]
Waiting for 5 seconds...
Export request pending. [id="JSOC_20190811_276", status=1]
Waiting for 5 seconds...
Export request pending. [id="JSOC_20190811_276", status=1]
Waiting for 5 seconds...
Export request pending. [id="JSOC_20190811_276", status=1]
Waiting for 5 seconds...
Export request pending. [id="JSOC_20190811_276", status=1]
Waiting for

Skipping download of file hmi.sharp_720s.942.20111010_210000_TAI.bitmap.fits as it has already been downloaded. If you want to redownload the data, please set overwrite to True
Skipping download of file hmi.sharp_720s.942.20111010_220000_TAI.bitmap.fits as it has already been downloaded. If you want to redownload the data, please set overwrite to True
Skipping download of file hmi.sharp_720s.942.20111010_230000_TAI.bitmap.fits as it has already been downloaded. If you want to redownload the data, please set overwrite to True
Skipping download of file hmi.sharp_720s.942.20111011_000000_TAI.bitmap.fits as it has already been downloaded. If you want to redownload the data, please set overwrite to True
Skipping download of file hmi.sharp_720s.942.20111011_010000_TAI.bitmap.fits as it has already been downloaded. If you want to redownload the data, please set overwrite to True
Skipping download of file hmi.sharp_720s.942.20111011_020000_TAI.bitmap.fits as it has already been downloaded. If 

2493 URLs found for download. Full request totalling 110MB
[                                        ========================================
Export request pending. [id="JSOC_20190811_280", status=2]
Waiting for 0 seconds...
Request not found on server, 5 retries left.
Export request pending. [id="JSOC_20190811_280", status=6]
Waiting for 5 seconds...
Request not found on server, 4 retries left.
Export request pending. [id="JSOC_20190811_280", status=6]
Waiting for 5 seconds...
Export request pending. [id="JSOC_20190811_280", status=1]
Waiting for 5 seconds...
Export request pending. [id="JSOC_20190811_280", status=1]
Waiting for 5 seconds...
Export request pending. [id="JSOC_20190811_280", status=1]
Waiting for 5 seconds...
Export request pending. [id="JSOC_20190811_280", status=1]
Waiting for 5 seconds...
Export request pending. [id="JSOC_20190811_280", status=1]
Waiting for 5 seconds...
Export request pending. [id="JSOC_20190811_280", status=1]
Waiting for 5 seconds...
Export reques

2048 URLs found for download. Full request totalling 100MB
[                                        ========================================
Export request pending. [id="JSOC_20190811_283", status=2]
Waiting for 0 seconds...
Request not found on server, 5 retries left.
Export request pending. [id="JSOC_20190811_283", status=6]
Waiting for 5 seconds...
Request not found on server, 4 retries left.
Export request pending. [id="JSOC_20190811_283", status=6]
Waiting for 5 seconds...
Export request pending. [id="JSOC_20190811_283", status=1]
Waiting for 5 seconds...
Export request pending. [id="JSOC_20190811_283", status=1]
Waiting for 5 seconds...
Export request pending. [id="JSOC_20190811_283", status=1]
Waiting for 5 seconds...
Export request pending. [id="JSOC_20190811_283", status=1]
Waiting for 5 seconds...
Export request pending. [id="JSOC_20190811_283", status=1]
Waiting for 5 seconds...
Export request pending. [id="JSOC_20190811_283", status=1]
Waiting for 5 seconds...
Export reques

[                                        ========================================
Export request pending. [id="JSOC_20190811_287", status=2]
Waiting for 0 seconds...
Request not found on server, 5 retries left.
Export request pending. [id="JSOC_20190811_287", status=6]
Waiting for 5 seconds...
Request not found on server, 4 retries left.
Export request pending. [id="JSOC_20190811_287", status=6]
Waiting for 5 seconds...
Export request pending. [id="JSOC_20190811_287", status=1]
Waiting for 5 seconds...
Export request pending. [id="JSOC_20190811_287", status=1]
Waiting for 5 seconds...
Export request pending. [id="JSOC_20190811_287", status=1]
Waiting for 5 seconds...
Export request pending. [id="JSOC_20190811_287", status=1]
Waiting for 5 seconds...
Export request pending. [id="JSOC_20190811_287", status=1]
Waiting for 5 seconds...
Export request pending. [id="JSOC_20190811_287", status=1]
Waiting for 5 seconds...
Export request pending. [id="JSOC_20190811_287", status=1]
Waiting for 5

2613 URLs found for download. Full request totalling 109MB
[                                        ========================================
Export request pending. [id="JSOC_20190811_292", status=2]
Waiting for 0 seconds...
Request not found on server, 5 retries left.
Export request pending. [id="JSOC_20190811_292", status=6]
Waiting for 5 seconds...
Request not found on server, 4 retries left.
Export request pending. [id="JSOC_20190811_292", status=6]
Waiting for 5 seconds...
Export request pending. [id="JSOC_20190811_292", status=1]
Waiting for 5 seconds...
Export request pending. [id="JSOC_20190811_292", status=1]
Waiting for 5 seconds...
Export request pending. [id="JSOC_20190811_292", status=1]
Waiting for 5 seconds...
Export request pending. [id="JSOC_20190811_292", status=1]
Waiting for 5 seconds...
Export request pending. [id="JSOC_20190811_292", status=1]
Waiting for 5 seconds...
Export request pending. [id="JSOC_20190811_292", status=1]
Waiting for 5 seconds...
Export reques

Export request pending. [id="JSOC_20190811_294", status=1]
Waiting for 5 seconds...
Export request pending. [id="JSOC_20190811_294", status=1]
Waiting for 5 seconds...
Export request pending. [id="JSOC_20190811_294", status=1]
Waiting for 5 seconds...
Export request pending. [id="JSOC_20190811_294", status=1]
Waiting for 5 seconds...
Export request pending. [id="JSOC_20190811_294", status=1]
Waiting for 5 seconds...
Export request pending. [id="JSOC_20190811_294", status=1]
Waiting for 5 seconds...
Export request pending. [id="JSOC_20190811_294", status=1]
Waiting for 5 seconds...
Export request pending. [id="JSOC_20190811_294", status=1]
Waiting for 5 seconds...
Export request pending. [id="JSOC_20190811_294", status=1]
Waiting for 5 seconds...
Export request pending. [id="JSOC_20190811_294", status=1]
Waiting for 5 seconds...
Export request pending. [id="JSOC_20190811_294", status=1]
Waiting for 5 seconds...
Export request pending. [id="JSOC_20190811_294", status=1]
Waiting for 5 sec

Export request pending. [id="JSOC_20190811_298", status=1]
Waiting for 5 seconds...
Export request pending. [id="JSOC_20190811_298", status=1]
Waiting for 5 seconds...
Export request pending. [id="JSOC_20190811_298", status=1]
Waiting for 5 seconds...
Export request pending. [id="JSOC_20190811_298", status=1]
Waiting for 5 seconds...
1967 URLs found for download. Full request totalling 76MB
[                                        ========================================
Export request pending. [id="JSOC_20190811_299", status=2]
Waiting for 0 seconds...
Request not found on server, 5 retries left.
Export request pending. [id="JSOC_20190811_299", status=6]
Waiting for 5 seconds...
Export request pending. [id="JSOC_20190811_299", status=1]
Waiting for 5 seconds...
Export request pending. [id="JSOC_20190811_299", status=1]
Waiting for 5 seconds...
Export request pending. [id="JSOC_20190811_299", status=1]
Waiting for 5 seconds...
Export request pending. [id="JSOC_20190811_299", status=1]


1809 URLs found for download. Full request totalling 74MB
[                                        ========================================
Export request pending. [id="JSOC_20190811_304", status=2]
Waiting for 0 seconds...
Request not found on server, 5 retries left.
Export request pending. [id="JSOC_20190811_304", status=6]
Waiting for 5 seconds...
Export request pending. [id="JSOC_20190811_304", status=1]
Waiting for 5 seconds...
Export request pending. [id="JSOC_20190811_304", status=1]
Waiting for 5 seconds...
Export request pending. [id="JSOC_20190811_304", status=1]
Waiting for 5 seconds...
Export request pending. [id="JSOC_20190811_304", status=1]
Waiting for 5 seconds...
Export request pending. [id="JSOC_20190811_304", status=1]
Waiting for 5 seconds...
Export request pending. [id="JSOC_20190811_304", status=1]
Waiting for 5 seconds...
Export request pending. [id="JSOC_20190811_304", status=1]
Waiting for 5 seconds...
Export request pending. [id="JSOC_20190811_304", status=1]


Export request pending. [id="JSOC_20190811_309", status=1]
Waiting for 5 seconds...
Export request pending. [id="JSOC_20190811_309", status=1]
Waiting for 5 seconds...
Export request pending. [id="JSOC_20190811_309", status=1]
Waiting for 5 seconds...
Export request pending. [id="JSOC_20190811_309", status=1]
Waiting for 5 seconds...
Export request pending. [id="JSOC_20190811_309", status=1]
Waiting for 5 seconds...
Export request pending. [id="JSOC_20190811_309", status=1]
Waiting for 5 seconds...
Skipping download of file hmi.sharp_720s.1574.20120427_000000_TAI.bitmap.fits as it has already been downloaded. If you want to redownload the data, please set overwrite to True
Skipping download of file hmi.sharp_720s.1582.20120427_000000_TAI.bitmap.fits as it has already been downloaded. If you want to redownload the data, please set overwrite to True
Skipping download of file hmi.sharp_720s.1596.20120427_000000_TAI.bitmap.fits as it has already been downloaded. If you want to redownload t

In [4]:
response = Fido.search(
    attrs.jsoc.Time(start, end),
    attrs.jsoc.Notify('hsmgroupnasa@gmail.com'),
    attrs.jsoc.Series('hmi.Sharp_720s'),
    attrs.jsoc.Segment('bitmap'),
    attrs.Sample(time_interval.total_seconds() * u.s)
    )
response

HTTPError: HTTP Error 500: Internal Server Error

## This Cell iterates over all downloaded sharp files and extracts the given keywords in order on to a text file, which is saved as data.txt in the workdir

In [5]:
warnings.simplefilter("ignore")#.verify('fix') produces many warnings which will lag the jupyter notebook

keywords = ['HARPNUM','T_REC','NAXIS1','NAXIS2','CDELT1','CDELT2','IMCRPIX1','IMCRPIX2','LAT_FWT','LON_FWT','NPIX']#Keywords in order to be saved
filenames = os.listdir(sharp_dir)
filename = 'data.txt'

data = open(workdir + filename,"w+")
line = ''
for keyword in keywords:
    line += keyword + ' '
line += 'Filename'
data.write(line + "\n")
for filename in filenames:
    line = ''
    hdul = fits.open(sharp_dir + filename)
    hdul.verify('fix')
    for keyword in keywords:
        line += str(hdul[1].header[keyword]) + ' '
    line += filename
    data.write(line + '\n')
    hdul.close()
data.close()